In [4]:
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)
#dealing with midi   files
from music21 import * 


#array   processing
import numpy as np     
import os

#random   number generator
import random         

#keras   for building deep learning model
from keras.layers import * 
from keras.models import *
import keras.backend as K
def read_midi(file):
  notes=[]
  notes_to_parse = None

 #parsing a midi   file
  midi = converter.parse(file)
 #grouping based on   different instruments
  s2 = instrument.partitionByInstrument(midi)

 #Looping over all   the instruments
  for part in s2.parts:
 #select elements   of only piano
    if 'Piano' in str(part): 
      notes_to_parse = part.recurse() 
 #finding   whether a particular element is note or a chord
      for element in notes_to_parse:
        if isinstance(element, note.Note):
          notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
          notes.append('.'.join(str(n) for n in element.normalOrder))

  return notes

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html
Using TensorFlow backend.


In [5]:
path="C:/Users/lulux/Desktop/music/"
#read   all the filenames
files=[i for i in os.listdir(path) if i.endswith(".mid")]

#reading   each midi file
all_notes=[]
for i in files:
  all_notes.append(read_midi(i))
  print(i)

#notes   and chords of all the midi files
notes = [element for notes in all_notes for element in notes]

schubert_D850_1.mid
schubert_D850_2.mid
schubert_D850_3.mid
schubert_D850_4.mid
schubert_D935_1.mid
schubert_D935_2.mid
schubert_D935_3.mid
schubert_D935_4.mid
schub_d760_1.mid
schub_d760_2.mid
schub_d760_3.mid
schub_d760_4.mid
schub_d960_1.mid
schub_d960_2.mid
schub_d960_3.mid
schub_d960_4.mid
schuim-1.mid
schuim-2.mid
schuim-3.mid
schuim-4.mid
schumm-1.mid
schumm-2.mid
schumm-3.mid
schumm-4.mid
schumm-5.mid
schumm-6.mid
schu_143_1.mid
schu_143_2.mid
schu_143_3.mid


In [6]:
#length   of a input sequence
no_of_timesteps = 128      

n_vocab = len(set(notes))  
pitch = sorted(set(item for item in notes))  

#no. of   unique notes
note_to_int = dict((note, number) for number, note in enumerate(pitch))  
#all   the unique notes
X = []
y = []
for notes in all_notes:
  for i in range(0, len(notes) - no_of_timesteps, 1):
    input_ = notes[i:i + no_of_timesteps]
    output = notes[i + no_of_timesteps]
    X.append([note_to_int[note] for note in input_])
    y.append(note_to_int[output])
X = np.reshape(X, (len(X), no_of_timesteps, 1))
#normalizing   the inputs


In [7]:
X = X / float(n_vocab)
print(X)
def lstm():
  model = Sequential()
  model.add(LSTM(128,return_sequences=True))
  model.add(LSTM(128))
  model.add(Dense(256))
  model.add(Activation('relu'))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
  return model
K.clear_session()

def simple_wavenet():
  no_of_kernels=64
  num_of_blocks= int(np.sqrt(no_of_timesteps)) - 1 

  model = Sequential()
  for i in range(num_of_blocks):
    model.add(Conv1D(no_of_kernels,3,dilation_rate=(2**i),padding='causal',activation='relu'))
  model.add(Conv1D(1, 1, activation='relu', padding='causal'))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(n_vocab, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
  return model


[[[0.3125    ]
  [0.25986842]
  [0.75328947]
  ...
  [0.99013158]
  [0.83881579]
  [0.94736842]]

 [[0.25986842]
  [0.75328947]
  [0.30263158]
  ...
  [0.83881579]
  [0.94736842]
  [0.83881579]]

 [[0.75328947]
  [0.30263158]
  [0.75328947]
  ...
  [0.94736842]
  [0.83881579]
  [0.90460526]]

 ...

 [[0.30263158]
  [0.51644737]
  [0.95065789]
  ...
  [0.71710526]
  [0.71710526]
  [0.71710526]]

 [[0.51644737]
  [0.95065789]
  [0.76973684]
  ...
  [0.71710526]
  [0.71710526]
  [0.71710526]]

 [[0.95065789]
  [0.76973684]
  [0.84868421]
  ...
  [0.71710526]
  [0.71710526]
  [0.71710526]]]


In [ ]:
from keras.callbacks import ModelCheckpoint
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]
    model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)

model=simple_wavenet()
netinput=X
y = np.array(y)
netoutput=y
train(model,netinput,netoutput)

In [ ]:
#mc = tf.keras.callbacks.ModelCheckpoint('model{epoch:03d}.h5', save_weights_only=False, period=50)

In [ ]:
#model = simple_wavenet()


In [ ]:
#y = np.array(y)
#model.fit(X,y, epochs=200, batch_size=1280,callbacks=[mc])

In [8]:
def generate_music(model, pitch, no_of_timesteps, pattern):

    int_to_note = dict((number, note) for number, note in enumerate(pitch))
    prediction_output = []

 # generate 50   elements
    for note_index in range(50):
 #reshaping   array to feed into model
        input_ = np.reshape(pattern, (1, len(pattern), 1))
 #predict   the probability and choose the maximum value
        proba = model.predict(input_, verbose=0)
        index = np.argmax(proba)

        pred = int_to_note[index]
        prediction_output.append(pred)
        pattern = list(pattern)
        pattern.append(index/float(n_vocab))
 #leave   the first value at index 0
        pattern = pattern[1:len(pattern)]
        print("generate succeed")

    return prediction_output


In [9]:
def convert_to_midi(prediction_output):
    offset = 0
    output_notes = []

 # create note and   chord objects based on the values generated by the model
    for pattern in prediction_output:
 #   pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
 #   pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

 #   Specify duration between 2 notes
        offset+= 0.5
       # offset += random.uniform(0.5,0.9)

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music1.mid')
    print("convert succeed")



In [10]:
#Select   random chunk for the first iteration
start = np.random.randint(0, len(X)-1)
pattern = X[start]
#load   the best model
model=load_model('weights-improvement--{epoch:02d}-{loss:.4f}-bigger.hdf5')

OSError: Unable to open file (unable to open file: name = 'weights-improvement--{epoch:02d}-{loss:.4f}-bigger.hdf5', errno = 22, error message = 'Invalid argument', flags = 0, o_flags = 0)

In [ ]:
#generate   and save music
music = generate_music(model,pitch,no_of_timesteps,pattern)
print("done")
convert_to_midi(music)
print("done1")